In [57]:
# version: v5-02-2023
# Changes or added features:
#   1. Set condition for data collection from whole dataset
#       should equal to the time of gap index.

# You can call this jupyter-notebook script using [import import_ipynb]
# To use this method, install the library "import-ipynb"
# >> ! pip install import-ipynb

#from netCDF4 import Dataset
#import xarray as xr
import csv
import pandas as pd
import numpy as np
#import pyresample
import matplotlib.pyplot as plt
import glob
import time
import math
import os
from math import isnan, sqrt
#import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.dates as mdates
import statsmodels.api as sm
import json
from datetime import datetime




In [58]:
# Important !!!!!
# Do not forget this to update before running the entire script.
# always use index 0 --> for PV Production data; 
# priority variables will determine by its order from index 1 --> to last index; 
# where index 1 is HIGH priority.
# example => ['production','test1', 'test2',...,'test-n']
PRIORITY_VARIABLE = ['Production','SWRRPrime','SWR']


# Important !!!!!
# define an index header-name; by default, set as 'index'
INDEX = 'Index'

# define a datetime header-name; by default set as 'Date-Time'
DATETIME = "Date-Time"

# define csv filename id (Optional but recommended. For naming purposes only)
REPORT_ID = "SWTTest"

# Define the PATH where your CSV as  Input file, is located.
SOURCE_PATH = "bin/All Datasets.csv"

In [59]:
# Important !!!!!
#++++++++++++++++++++++++++++++++++
# Class for Global Variable
#++++++++++++++++++++++++++++++++++

# Other global variables
finalListArray = [] 

# ++++++++ FOR DEBUGGING ++++++++++++++++++++++
# for debugging and data review purpose only.
passed_possible_values = []
# +++++++++++++++++++++++++++++++++++++++++++++

class globalVariable():
    def directory(self):
        return SOURCE_PATH

 
    def newDirectory(self):
 
        dir, filename = "" , REPORT_ID + "_gapsfilled.csv"  
        return dir, filename


    def useableVar(self):
        percent = 0.5 # 0.1 == 10% ; this is a limit for increment_by variable
        increment_by = 0.01
        return percent, increment_by

In [60]:
# get an average values from a dictionary with integer/float datatype
def average(data, field):
    values = [float(item[field]) for item in data]
    return sum(values) / len(values)

In [61]:
def reconstruct_dictionary(dic):
    dataset={}

    param=[]
    for data in dic:
        for key, val in data.items():
            param.append(key)
            
    param = set(param)

    for p in param:
        values=[]
        for data in dic:
            for k, v in data.items():
                if k == p:
                    values.append(str(v).replace("\n",""))

        dataset[p] = values

    return dataset

In [62]:


#++++++++++++++++++++++++++++++++++
# Class for multi-dimentional Array
#++++++++++++++++++++++++++++++++++
class multiArray:
    def __init__(self, index, percent, data):
        self.index = index
        self.percent = percent
        self.data = data
        
        


class showItems():
    def showlist(self, array):
        for f in array:            
            print("Get: prod=" + str(round(float(f.param1),3)) +" | SWR="+ str(round(float(f.param2),3)) +" | t2m="+ str(round(float(f.param3),3)) +" | tp="+ str(round(float(f.param4),3)) +" | rh="+ str(round(float(f.param5),3)))
            #return f.param1, f.param2, f.param3, f.param4, f.param5

    def getlist(self, array):
        for f in array:
            #print("Get: " + str(f.param2) +" | "+ str(f.param3) +" | "+ str(f.param4) +" | " "+ str(f.param5) +" | ")
            return f.param1, f.param2, f.param3, f.param4, f.param5


#++++++++++++++++++++++++++++++++++
# Class for fn()
#++++++++++++++++++++++++++++++++++
class fn():

    def dateToUnix(self, x):
       
        dd = x.split("/")
    
        date, month, year = int(dd[0]), int(dd[1]), int(dd[2]) #STARTING DATE

        unixUTC8 = datetime(year, month, date).timestamp()
        return unixUTC8

    def unixToDate(self, dd):
        dd = datetime.fromtimestamp(dd)
        x = str(dd.strftime("%x")) # text_format: mm/dd/yy
        x = x.split("/")
        xdate = x[0] +"-"+ x[1] +"-20"+ x[2]
        return xdate


    def convert24hr(self, str1):
        
        # Checking if last two elements of time
        # is AM and first two elements are 12
        am_str = "am"
        pm_str = "pm"
        
        if str1[-2:] == am_str and str1[:2] == "12":
            return "00" + str1[2:-2]
            
        # remove the AM    
        elif str1[-2:] == am_str:
            return str1[:-2]
        
        # Checking if last two elements of time
        # is PM and first two elements are 12   
        elif str1[-2:] == pm_str and str1[:2] == "12":
            return str1[:-2]
            
        else:
            
            # add 12 to hours and remove PM
            # 00:00:00 AM
            #print(str(str1[:1]) +" || "+ str(str1[2:8]))
            return str(int(str1[:1]) + 12) +":"+ str1[2:8]


    def getRangeSamples(self, possible_items_within_range, neg, pos, key_name):
        temp = []
        
        for fff in possible_items_within_range : # All items from reloopDataline() is => possible_items_within_range[]
            for i in range(len(PRIORITY_VARIABLE)):
                var_name = PRIORITY_VARIABLE[i]
                if key_name == var_name :
                    v = float(fff[var_name])

                    # This "v" value will check if it will passed the condition
                    if  v >= neg and v <= pos:
                        temp.append(fff)
        return temp


    def prevDate(self, dd):
        count = 1
        prev_week = dd
        while count <= 14:
            prev_week -= 86400 # 86400s = 1 day or 24 hrs
            count += 1
        return prev_week
    
    def advDate(self, dd):
        count = 1
        adv_week = dd
        while count <= 14:
            adv_week += 86400
            count += 1
        return adv_week


    
    
   
        

#++++++++++++++++++++++++++++++++++
# Class for dataframe
#++++++++++++++++++++++++++++++++++

class dictionaryFrame():
    def __init__(self, data):
        self.__dictionary = data

    def create_var_as_dictionary(self, columns, values):
        data={}
        for k in range(len(columns)):
            if columns[k] == INDEX:
                data[INDEX] = values[k]
            else:
                data[columns[k]] = str(values[k]).replace("\n","")

        self.__dictionary = data
    
    def get_dictionary_dataframe(self):
        return self.__dictionary

#++++++++++++++++++++++++++++++++++
# Class for processing
#++++++++++++++++++++++++++++++++++
class processWithCorrection:

    # get the first list from +/- 1% range :: irridiance :: but when irridiance is zero, product data is zero
    # from that first list, get its equivalent, humidity, temperature, etc.. :: get the nearest possible values 
    # if all conditions are meet, get the equivalent product data :: that irridiance <> 0
        
    def readCSV(self):
        global rowLength

        
        groFile =  globalVariable().directory()

        dicFrame = dictionaryFrame(None)

        # read csv using pandas; to get header
        testdata= pd.read_csv(groFile)
        df_cols = testdata.columns

        with open(groFile,"r",encoding='utf-8') as f:
            f.readline()
            
            #index,datetime,ws,wd,rh,t2m,hcc,lcc,mcc,ssrd,tcc,tp,SWR,production  = ([] for i in range(14))
            for line in f:
                lineSplit = line.split(",") 
                #print("Monitor loop by row =>", line)
                # to prevent some cases where string array got 1 ONLY; take note na yung last variable has "\n" text
                if len(lineSplit) == len(df_cols): 
                    # use class to make columns as variables for each values in a row
                    dicFrame.create_var_as_dictionary(df_cols,lineSplit)
                    dic_vars = dicFrame.get_dictionary_dataframe()
                    #print("=>", dic_vars)
                    try:
                        self.repairData(dic_vars)
                    except Exception as e:
                        return False
        
        if len(finalListArray) > 0:
            for data in finalListArray:
                print(data.index, data.percent, data.data, "\n")
            return True
        else:
            return False
                    
  
    def repairData(self, data_each_row):
        

        # Make constant local variable for production and test 1
        PVPROD = PRIORITY_VARIABLE[0]
        TEST1 = PRIORITY_VARIABLE[1]

        production=None
        try:
            # use float to determine if production value is integer or numbers; 
            # otherwise, it is an empty or invalid value; considered as NULL or NA
            production = float(str(data_each_row[PVPROD]))
        except ValueError as e:
            if str(e) == "could not convert string to float: ''" :
                production = None
        
        if production is None: # condition to filter empty data 
            #print("!!! FOUND EMPTY Production !!!")
            limit_percent, inc_by = globalVariable().useableVar() 
            
            lock = 0 # purpose nito is to stop the loop pag lock == 1
            while lock == 0 :
                p1_neg, p1_pos = self.getNegPos(data_each_row[TEST1], inc_by) # from first parameter conditioning irridiance, obtain min and max value
              
                
                # This dataGapList array will record only each index with empty pv production.
                # dataGapList.append(singleArray(data_each_row.index, data_each_row[TEST1], t2m, tp, rh)) # obtain daw rows which irrradiance passes the min and max conditions 

                if(inc_by >= limit_percent):
                    lock=1  # break the while loop if increment reaches the limit-percent; It means that no possible values found during incremental scanning;
                    break
            
                # The datarow dictionary has an empty production value.
                lock, inc_by = self.reloopDataline(data_each_row, p1_neg, p1_pos, inc_by)
                #print(str(percent) +" >> Increment percentage range")

                #time.sleep(0.1) # give time to sleep 0.01 seconds before looping back
            
    def getNegPos(self, param, percent):
        if param == "":
            #print("return " + str(Time))
            return 0,0
        param_neg = float(param) - (float(param) * percent)  #2.55 - (2.55 * 0.1) lower bound
        param_pos = float(param) + (float(param) * percent) #2.55 + (2.55 * 0.1) upper bound 

        return param_neg, param_pos



    def reloopDataline(self, datarow, p1_neg, p1_pos, percent):
        global passed_possible_values
        
        # take note yung "datarow" is first row of data pa lang ito
        groFile =  globalVariable().directory()

        # read csv using pandas; to get header
        testdata= pd.read_csv(groFile)
        csv_columns = testdata.columns
        # this variable will store all possible values for each loop
        possible_values = []
        
        # call class for converting list-of-items from csv to dictionary format.
        dicFrame = dictionaryFrame(None)

        # Make constant local variable for production and test 1
        PVPROD = PRIORITY_VARIABLE[0]
        TEST1 = PRIORITY_VARIABLE[1]

        # ngayon i open ulit natin ang parehong file, para sa panibagong splitting (refers to "," delimeter) ng data per row.
        with open(groFile,"r",encoding='utf-8') as f:
            
            f.readline()
            for line in f:
                
                ls = line.split(",") # splitting (refers to "," delimeter)

                # create variable "production", "test1=0"; None value as default
                production=None
                test1 = 0
                __datetime=None

                # get the PV-Production and test1 values from "ls" variable.
                for k in range(len(csv_columns)):
                    if csv_columns[k] == PVPROD:
                        production = ls[k]
                    if csv_columns[k] == TEST1:
                        test1 = ls[k]
                    if csv_columns[k] == DATETIME:
                        __datetime = ls[k]

                
                if len(ls) == len(csv_columns):
                    
                    # use class to make columns as variables for each values in a row
                    dicFrame.create_var_as_dictionary(csv_columns,ls)
                    dic_vars = dicFrame.get_dictionary_dataframe()

                    try:
                        production = float(str(production))
                    except ValueError as e:
                        if str(e) == "could not convert string to float: ''" :
                            production = None
                          
                    # 1st splitting to map empty production i.e if production == "" or production == "-" or production == "x":
                    # 2nd splitting to map not empty production
                    if production != "" and production != "-" and production is not None:
                        
                        # P1 => means, parameter 1; this time it is an SWR.
                        # pag na detect yung row na data na HINDI empty, kukunin yung value ng TEST1
                        p1 = float(test1) 
                        
                        # use this function [ fn.convert24hr(datetime) ] ONLY if time format has AM/PM
                        __date_time = __datetime.split(" "); #print(datarow)
                        __date_time_from_gap = str(datarow[DATETIME]).split(" ") # take note na "datarow" array ito.
                        
                        '''
                        if __date_time_from_gap[1] == __date_time[1]:
                            if  rh >= p1_neg and rh <= p1_pos:
                                #get data of possible values within the given range => [1% to 5%]
                                possible_values.append(singleArray(production, rh, t2m, ssrd, tp))
                        
                            else: 
                                #print(">>>>>>>> pass!")
                                pass  
                        '''
                        
                        #date1 = fn().dateToUnix(yt[0]) # noong na split, bale yyy/mm/dd ito
                        #date2 = fn().dateToUnix(xt[0]) # noong na split, bale yyy/mm/dd ito; date2 ang ginamit sa condition sa if-then-else below
                        #prev_week = fn().prevDate(date1) # get the previous week
                        #adv_week = fn().advDate(date1)  # get the latest week
                        
                        # Commented last January 25, 2023 -> the previous week and advance week conditioning will be DISABLED.
                        # Because all dataset will be used instead.
                        #if date2 >= prev_week and date2 <= adv_week : -> DISABLED condition
                        
                        # New filter added on February 3, 2023;
                        # added new filter for time; where the condition is that the time of gap index 
                        # must be equal to the collected values;
                        #print(str(__date_time_from_gap[1]),str(__date_time[1]))
                        #if str(__date_time_from_gap[1]) == str(__date_time[1]):
                        
                        if  p1 >= p1_neg and p1 <= p1_pos: # 1st condition using test1 (irradiance)
                            # create temperorary dictionary var
                            temp_dic = {}
                            for var in PRIORITY_VARIABLE:
                                for k in range(len(csv_columns)):
                                    if csv_columns[k] == var:
                                        temp_dic[var] = ls[k]
                            
                            possible_values.append(temp_dic); temp_dic=None  

                            # ++++++++ FOR DEBUGGING ++++++++++++++++++++++
                            # for debugging and data review purpose only.
                            temp_dic={}
                            for k in range(len(csv_columns)):
                                temp_dic[csv_columns[k]] = ls[k]
                            passed_possible_values.append(temp_dic); temp_dic=None    
                            # ++++++++ FOR DEBUGGING ++++++++++++++++++++++                 
                            

        # Here, using a quantifyArray() function, all possible_values will be check one,by one from a datarow-> a data with empty pv production.
        lock, percent = self.quantifyArray(datarow,possible_values, percent)
        #datarow refers to column label, possible_values are data that passed irradiance condition,percent
        return lock, percent


    def quantifyArray(self, datarow, possible_items_within_range, percent):
        global finalListArray
        
        # yung "inc_by" ang ginamit ko lang dito; para ma increment lang yung percent variable dito.
        percent_not_used_here, inc_by = globalVariable().useableVar() 
        
        # create temp (temporary) array => temp = []
        temp = []
        for i in range(len(PRIORITY_VARIABLE)):
            # skip the index-0
            if i>0:
                key_name = PRIORITY_VARIABLE[i]
                handle_value = datarow.get(key_name)
                down_range, up_range = self.getNegPos(handle_value, percent)

                # get the samples of those that satisfies the condition.
                temp = fn().getRangeSamples(possible_items_within_range, down_range, up_range, key_name) # ito yung sa t2m na range
                
                if len(temp) == 0 :
                    percent = percent + inc_by
                    return 0, percent 
                    # so pag wala siyang nakuhang data or walang value yung temp, babalik siya para mag loop ulit
                    # kukuha ulit siya ng ibang values by increasing the percent value
                    # so balik sa first step, which is the 1st condition for possible values
        
        # if nakaabot dito yung filtering data na nakuha from possible values,
        # na satisfy niya lahat ng filters, proceed na sa final steps of this function
        # check if data is need for averaging.
        if len(temp) > 1:
            
            finale = {}

            for field in PRIORITY_VARIABLE:
                result = average(temp, field)
                finale[field] = result
            
            # add [] on finale as => [finale]; 
            # This is to avoid key_error=0 from the computeError().realignList()
            finalListArray.append(multiArray(datarow.get(INDEX), percent, [finale]))
        else:
            finalListArray.append(multiArray(datarow.get(INDEX), percent, temp))

        '''
        print("+"*10, "Possible values Collected", "+"*10)
        print(possible_items_within_range,"\n")
        print("+"*10, "Filtered values Collected", "+"*10)
        print(temp,"\n")
        print("+"*10, "Possible values Collected", "+"*10)
        for data in finalListArray:
            print(data.index, data.percent, data.data, "\n")
        #time.sleep(60) # 60 seconds or 1 minute delay
        '''

        return 1, percent # END the LOOP HERE
        #END HERE
        #+++++++++++++++++++++++++++++++++++++++++++++


        

    





  

In [63]:
class computeError():

    def percentError(self, index, gen, orig_data):
        orig = orig_data
        error = 0
        
        orig_num = 0
        for n in orig:
            if str(index) == str(n.index):
                if float(n.data) > 0:
                    error = abs( float(n.data) - float(gen)) / float(n.data)
                else:
                    error = 0
                orig_num = n.data
        
        return orig_num, error  
    

    def realignList(self):
        dir, filename = globalVariable().newDirectory()
        
        # Note: a finalistArray has the following key items;
        # def __init__(self, index, percent, data):
        #    self.index = index
        #    self.percent = percent
        #    self.data = data

        # this array will be used only for orig_vs_generated.csv report
        index_report, generated_report, percent_report, datetime_report =([] for i in range(4))
        
        # get the path of input dataset
        pathFile = globalVariable().directory()
        orig_dataset = pd.read_csv(pathFile)

        # get the column header; and the row data; from dataset.
        col_header = orig_dataset.columns
        row_values = orig_dataset.values.tolist()


        # extract line-per-line data from array of row-values
        for data in row_values:
            # Now, extract final-list-of-dataset (gap-filled)
            for finale in finalListArray :
        
                # determine the data of equivalent header-name by looping col-header.
                for k in range(len(col_header)):
                    
                    # If TEST1 (HIGH PRIORITY variables) == 0; Production must be ZERO.
                    if col_header[k] == PRIORITY_VARIABLE[1]:
                        if data[k] <=0:
                            # get the production; and set to ZERO.
                            # determine the data of equivalent header-name by looping col-header.
                            for w in range(len(col_header)):
                                if col_header[w] == PRIORITY_VARIABLE[0]:
                                    data[w] = 0

                    # check for index from original-dataset vs final-list-of-dataset (gap-filled)
                    # this will determine the row-id or address of gap-filled from original dataset.
                    if col_header[k] == INDEX:
                        if int(data[k]) == int(finale.index):
                            #print(col_header[k], data[k], "=>", finale.data)

                            # init the var for irradiance-related variables;
                            get_test1 = None
                            # get the irradiance-related variables;
                            # determine the data of equivalent header-name by looping col-header.
                            for w in range(len(col_header)):
                                if col_header[w] == PRIORITY_VARIABLE[1]:
                                    get_test1 = data[w]

                            if get_test1 is not None:
                                # get the production;
                                # determine the data of equivalent header-name by looping col-header.
                                for w in range(len(col_header)):
                                    if col_header[w] == PRIORITY_VARIABLE[0]:
                                        # Replace the empty value of original-dataset from a gap-filled-dataset value
                                        # But only the 1st PRIORITY VARIABLE (which is the production)
                                        # Before that, check if TEST-1 (HIGH Priority weather condition) == ZERO; then production must be ZERO.
                                        if get_test1 <= 0:
                                            data[w] = 0
                                        else:
                                            data[w] = finale.data[0][PRIORITY_VARIABLE[0]]
                            else:
                                # cancel the whole process; then return false.
                                return False
                            
                            # init the var for datetime variables;
                            get_datetime = None
                            # get the value of datetime
                            for z in range(len(col_header)):
                                if col_header[z] == DATETIME:
                                    get_datetime = data[z]

                            index_report.append(finale.index)
                            generated_report.append(finale.data[0][PRIORITY_VARIABLE[0]])
                            percent_report.append(finale.percent)
                            datetime_report.append(get_datetime)
        
        
        df_gapfilled = pd.DataFrame(row_values, columns=col_header)
                    
        df_report = pd.DataFrame({'index': index_report,
                            'datetime': datetime_report,
                            'generated_values':generated_report,
                            'percent_range':percent_report
                            })

       
        __path = dir + filename
        df_gapfilled.to_csv(__path, index=None)

        __path = dir + REPORT_ID + "_generated_gapfilled_report.csv"
        df_report.to_csv(__path, index=None)

        # ++++++++ FOR DEBUGGING ++++++++++++++++++++++
        # for debugging and data review purpose only.
        dic = reconstruct_dictionary(passed_possible_values)
        df = pd.DataFrame(dic)
        __path = dir + REPORT_ID + "_all_possiblevalues_report.csv"
        df.to_csv(__path, index=None)
        # ++++++++ FOR DEBUGGING ++++++++++++++++++++++

        # the whole process is successful; then return true.
        return True
        # ------------------------------------------------
        
        

       


In [64]:
def main():
     
    success = processWithCorrection().readCSV()
    if success:
        success = computeError().realignList()

        if success:
            print("Finished! Gap filling processing is successful.")
        else:
            print("Something went wrong during the realigning of samples. Please inspect 'computeError().realignList()'.")
    else:
        print("Something went wrong during processing with 'processWithCorrection().readCSV()'. Please inspect your dataset.")
    



if __name__ == "__main__":
    main()

145 0.2900000000000001 [{'Production': '1103.680', 'SWRRPrime': '31.450', 'SWR': '28.608'}] 

152 0.07 [{'Production': '1103.680', 'SWRRPrime': '31.450', 'SWR': '28.608'}] 

377 0.01 [{'Production': 1091.9928571428572, 'SWRRPrime': 618.074, 'SWR': 598.606}] 

378 0.01 [{'Production': 1607.63975, 'SWRRPrime': 851.5625000000002, 'SWR': 824.740125}] 

379 0.01 [{'Production': 1892.105105263158, 'SWRRPrime': 992.8966315789473, 'SWR': 961.6224736842104}] 

631 0.18000000000000002 [{'Production': '1366.470', 'SWRRPrime': '13.413', 'SWR': '11.000'}] 

632 0.38000000000000017 [{'Production': '1366.470', 'SWRRPrime': '13.413', 'SWR': '11.000'}] 

662 0.04 [{'Production': '602.736', 'SWRRPrime': '177.450', 'SWR': '165.875'}] 

663 0.060000000000000005 [{'Production': 685.693, 'SWRRPrime': 167.898, 'SWR': 144.804}] 

664 0.03 [{'Production': '397.584', 'SWRRPrime': '80.049', 'SWR': '72.817'}] 

672 0.13999999999999999 [{'Production': '1103.680', 'SWRRPrime': '31.450', 'SWR': '28.608'}] 

720 0.37